# Benchmarking

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import axes3d

### Linear regression with one variable (Regression between #people and #profit in 10k units)

In [2]:
data = np.loadtxt('data/pop-profit.txt', delimiter=',')

# print the shape of data
print(data.shape)

# print the first 10 elements, all columns
print(data[:10,:])

(97, 2)
[[ 6.1101 17.592 ]
 [ 5.5277  9.1302]
 [ 8.5186 13.662 ]
 [ 7.0032 11.854 ]
 [ 5.8598  6.8233]
 [ 8.3829 11.886 ]
 [ 7.4764  4.3483]
 [ 8.5781 12.    ]
 [ 6.4862  6.5987]
 [ 5.0546  3.8166]]


### Supervised Learning

**Non-vectorized hypothesis**
$$\large h_{\theta}(x) = \theta_{0} + \theta_{1}x, \hspace{1pt} \hspace{1cm} \textit{where x is a single sample} $$ 

**Vectorized hypothesis**
$$\large h_{\theta}(X) = X\theta$$

In [3]:
# add the column of ones for the intercept term in Linear Regression
X = np.c_[np.ones(data.shape[0]), data[:,0]]
y = np.c_[data[:,1]]

In [4]:
print(X.shape)
print(y.shape)

(97, 2)
(97, 1)


In [5]:
print(X[:5, :])
print(y[:5, :])

[[1.     6.1101]
 [1.     5.5277]
 [1.     8.5186]
 [1.     7.0032]
 [1.     5.8598]]
[[17.592 ]
 [ 9.1302]
 [13.662 ]
 [11.854 ]
 [ 6.8233]]


# Linear Regression Cost Function (theory recap)

**Linear Regression Model (Non-vectorized):**
$$\large \widehat{y}_{i} = h_{\theta}(x_{i}) = x_{i}\theta_1 + \theta_0$$

**Linear Regression Model (Vectorized):**
$$\large \widehat{y} = h_{\theta}(X) = X \theta$$

**Dimensionality Sanity Check:**
$$\large dim[\widehat y_{i}] = dim[x_{i}\theta_1 + \theta_0] = 1 \times n \cdot n\times 1 + 1\times1 = 1 \times 1$$

$$\large dim[\widehat y] = dim[X \theta] = m \times n \cdot n \times 1 = m \times 1$$
**Non-Vectorized Implementation:**
$$\large J(\theta) = \frac{1}{2m} \sum_{i = 1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})^2$$
**Vectorized Implementation:**
$$\large J(\theta) = \frac{1}{2m} (X\theta - y)^{T}(X\theta - y)$$

## Cost Function Vectorized Implementation

**Vectorized Implementation:**
$$\large J(\theta) = \frac{1}{2m} (X\theta - y)^{T}(X\theta - y)$$

In [6]:
def computeCostVectorized(X, y, theta=np.zeros((2, 1))):
    m = y.size
    J = 0

    start = time.time()
    h = X.dot(theta)
    
    # vectorized implementation
    J = 1/(2*m)*((h-y).T.dot(h-y))
    end = time.time()
    eta = end-start
    return(J, eta)

## Cost Function Loop-based Implementation with Numpy

**Linear Hypothesis (Non-vectorized):**
$$\large \widehat{y}_{i} = h_{\theta}(x_{i}) = x_{i}\theta_1 + \theta_0$$

**Non-Vectorized Implementation:**
$$\large J(\theta) = \frac{1}{2m} \sum_{i = 1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})^2$$

In [7]:
def computeCostLoopNumpy(X, y, theta=np.zeros((2, 1))):
    m = y.size
    J = 0
    h = np.zeros((m,1))
    
    start = time.time()
    
    # hypothesis evaluation loop-based implementation
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            #h[i] = h[i] + theta[j] * X[i,j]
            h[i] += theta[j] * X[i,j]
    
    # semi-vectorized implementation (Numpy)
    J = 1/(2*m)*np.sum(np.square(h-y))
    end = time.time()
    eta = end-start
    return(J, eta)

## Cost Function Custom Loop-based Implementation

**Linear Hypothesis (Non-vectorized):**
$$\large \widehat{y}_{i} = h_{\theta}(x_{i}) = x_{i}\theta_1 + \theta_0$$

**Non-Vectorized Implementation:**
$$\large J(\theta) = \frac{1}{2m} \sum_{i = 1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})^2$$

In [8]:
def computeCostLoopCustom(X, y, theta=np.zeros((2, 1))):
    m = y.size
    J = 0
    
    start = time.time()
    
    # non-vectorized implementation (Custom)
    for i in range(X.shape[0]):
        h = 0
        for j in range(X.shape[1]):
            h += theta[j] * X[i,j]
        J += (h - y[i])**2
    
    end = time.time()
    eta = end-start
    return(J/(2*m), eta)

## Comparison between different Cost function implementations in terms of execution time

In [9]:
count = 10000
eta_vec_list = []
eta_ln_list = []
eta_lc_list = []

for i in range(count):
    J_vec, eta_vec = computeCostVectorized(X, y)
    eta_vec_list.append(eta_vec)
    
    J_ln, eta_ln = computeCostLoopNumpy(X, y)
    eta_ln_list.append(eta_ln)
    
    J_lc, eta_lc = computeCostLoopCustom(X, y)
    eta_lc_list.append(eta_lc)
    
print(J_vec[0][0], J_ln, J_lc[0])

# check if the cost function implementations are correct
# the assert returns an error if the argument is evaluated as false
assert(round(J_vec[0][0], 3) == round(J_ln, 3) == round(J_lc[0], 3))

# compute the average of eta values in each list
mean_eta_vec = np.mean(eta_vec_list)
mean_eta_ln = np.mean(eta_ln_list)
mean_eta_lc = np.mean(eta_lc_list)

32.07273387745567 32.072733877455676 32.072733877455654


In [10]:
print("Cost function Vectorized value : {}".format(J_vec[0][0]))
print("Cost function Loop-based Numpy value : {}".format(J_ln))
print("Cost function Loop-based Custom value : {}".format(J_lc[0]))

Cost function Vectorized value : 32.07273387745567
Cost function Loop-based Numpy value : 32.072733877455676
Cost function Loop-based Custom value : 32.072733877455654


In [11]:
print("Compute Cost Vectorized (Average time over {} iterations) {} [ms] {} [µs]".format(count, round(mean_eta_vec*1000, 3), round(mean_eta_vec*1000000, 3)))
print("Compute Cost Loop-based Numpy (Average time over {} iterations) {} [ms] {} [µs]".format(count, round(mean_eta_ln*1000, 3), round(mean_eta_ln*1000000, 3)))
print("Compute Cost Loop-based Custom (Average time over {} iterations) {} [ms] {} [µs]".format(count, round(mean_eta_lc*1000, 3), round(mean_eta_lc*1000000, 3)))

Compute Cost Vectorized (Average time over 10000 iterations) 0.006 [ms] 5.831 [µs]
Compute Cost Loop-based Numpy (Average time over 10000 iterations) 0.601 [ms] 600.816 [µs]
Compute Cost Loop-based Custom (Average time over 10000 iterations) 0.748 [ms] 748.401 [µs]


# (Batch) Gradient Descent (theory recap)

**Non-vectorized hypothesis**
$$\large h_{\theta}(x) = \theta_{0} + \theta_{1} x$$

**Non-vectorized cost function**
$$\large J(\theta) = \frac{1}{2m} \sum_{i = 1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})^2$$

**Gradient of the cost function wrt the parameter $\theta_{j}$**
$$\large \frac{\partial J(\theta)}{\partial \theta_{j}} = \frac{1}{m} \sum_{i = 1}^{m} (h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_{j} \rightarrow \frac{1}{m} (X^T(X\theta - y)$$

**Standard Implementation of Gradient Descent (ved. slide 130)**
$$\large \textit{Repeat until convergence} \hspace{1cm} \theta_{j} := \theta_{j} - \frac{\alpha}{m}\sum_{i = 1}^{m} (h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_{j} \hspace{1cm} \textit{for} \hspace{1cm} \theta_{0}, \theta_{1}, \dots , \theta_{n}$$

**Vectorized Implementation of Gradient Descent** 
$$\large \theta = \theta - \frac{\alpha}{m} ((X\theta - y)^T X)^T = \theta - \frac{\alpha}{m} (X^T(X\theta - y)$$

## Gradient Descent Vectorized

**Vectorized Implementation of Gradient Descent** 
$$\large \theta = \theta - \frac{\alpha}{m} ((X\theta - y)^T X)^T = \theta - \frac{\alpha}{m} (X^T(X\theta - y)$$

In [12]:
def gradientDescentVectorized(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=1500, early = False):
    m = y.size
    J_history = np.zeros(num_iters)
    
    start = time.time()
    for iter in np.arange(num_iters):
        h = X.dot(theta)
        
        # ! simoultaneusly update all the parameters 
        # vectorized implementation
        theta = theta - alpha*(1/m)*(X.T.dot(h-y))
        J_history[iter], _ = computeCostVectorized(X, y, theta)
        
        # early stopping
        if (early == True) & (J_history[iter] == J_history[iter-1]):
            break
            
    end = time.time()
    eta = end - start
    return(theta.ravel(), J_history[J_history != 0], eta)

In [13]:
theta_vec_es, J_history_vec_es, eta_vec_es = gradientDescentVectorized(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=10000, early = True)
theta_vec_nes, J_history_vec_nes, eta_vec_nes = gradientDescentVectorized(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=10000, early = False)
#mean_eta_vec_es = np.mean(eta_vec_es)
#mean_eta_vec_nes = np.mean(eta_vec_nes)

In [14]:
print("Gradient Descent Vectorized (Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_vec_es,3)*1000, round(eta_vec_es,3)*1000000, J_history_vec_es.shape[0]))
print("Gradient Descent Vectorized (Without Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_vec_nes,3)*1000, round(eta_vec_nes,3)*1000000, J_history_vec_nes.shape[0]))

Gradient Descent Vectorized (Early Stopping) / Execution time 87.0 [ms]; 87000.0 [µs] over 8000 iterations
Gradient Descent Vectorized (Without Early Stopping) / Execution time 93.0 [ms]; 93000.0 [µs] over 10000 iterations


## Gradient Descent Loop-based with Numpy

**Linear Regression Model (Non-vectorized):**
$$\large \widehat{y}_{i} = h_{\theta}(x_{i}) = x_{i}\theta_1 + \theta_0$$

$$\large \textit{Repeat until convergence} \hspace{1cm} \theta_{j} := \theta_{j} - \frac{\alpha}{m}\sum_{i = 1}^{m} (h_{\theta}(x^{(i)})-y^{(i)})x^{(i)}_{j} \hspace{1cm} \textit{for} \hspace{1cm} \theta_{0}, \theta_{1}, \dots , \theta_{n}$$

In [15]:
def gradientDescentLoopNumpy(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=1500, early = False):
    m = y.size
    J_history = np.zeros(num_iters)
    
    start = time.time()
    for iter in np.arange(num_iters):
        
        h = np.zeros((m,1))
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                h[i] += theta[j] * X[i,j]

        # ! simoultaneusly update all the parameters
        for j in range(X.shape[1]):
            gradient = 0
            for i in range(X.shape[0]):
                gradient += (h[i]-y[i])*X[i,j]
            theta[j] = theta[j] - alpha*(1/m)*gradient

        J_history[iter], _ = computeCostLoopNumpy(X, y, theta)
        
        # early stopping
        if (early == True) & (J_history[iter] == J_history[iter-1]):
            break
            
    end = time.time()
    eta = end - start
    return(theta.ravel(), J_history[J_history != 0], eta)

In [16]:
theta_ln_es, J_history_ln_es, eta_ln_es = gradientDescentLoopNumpy(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=10000, early = True)
theta_ln_nes, J_history_ln_nes, eta_ln_nes = gradientDescentLoopNumpy(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=10000, early = False)
#mean_eta_ln_es = np.mean(eta_ln_es)
#mean_eta_ln_nes = np.mean(eta_ln_nes)

In [17]:
print("Gradient Descent Loop-based Numpy (Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_ln_es,3)*1000, round(eta_ln_es,3)*1000000, J_history_ln_es.shape[0]))
print("Gradient Descent Loop-based Numpy (Without Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_ln_nes,3)*1000, round(eta_ln_nes,3)*1000000, J_history_ln_nes.shape[0]))

Gradient Descent Loop-based Numpy (Early Stopping) / Execution time 14149.0 [ms]; 14149000.0 [µs] over 7915 iterations
Gradient Descent Loop-based Numpy (Without Early Stopping) / Execution time 17811.0 [ms]; 17811000.0 [µs] over 10000 iterations


## Gradient Descent Loop-based Custom implementation

In [18]:
def gradientDescentLoopCustom(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=1500, early = False):
    m = y.size
    J_history = np.zeros(num_iters)
    
    start = time.time()
    for iter in np.arange(num_iters):
        
        h = np.zeros((m,1))
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                h[i] += theta[j] * X[i,j]

        # ! simoultaneusly update all the parameters
        for j in range(X.shape[1]):
            gradient = 0
            for i in range(X.shape[0]): 
                gradient += (h[i]-y[i])*X[i,j]
            theta[j] = theta[j] - alpha*(1/m)*gradient

        J_history[iter], _ = computeCostLoopCustom(X, y, theta)
        
        # early stopping
        if (early == True) & (J_history[iter] == J_history[iter-1]):
            break
            
    end = time.time()
    eta = end - start
    return(theta.ravel(), J_history[J_history != 0], eta)

In [19]:
theta_lc_es, J_history_lc_es, eta_lc_es = gradientDescentLoopCustom(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=10000, early = True)
theta_lc_nes, J_history_lc_nes, eta_lc_nes = gradientDescentLoopCustom(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=10000, early = False)
#mean_eta_lc_es = np.mean(eta_lc_es)
#mean_eta_lc_nes = np.mean(eta_lc_nes)

In [20]:
print("Gradient Descent Loop-based Custom (Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_lc_es,3)*1000, round(eta_lc_es,3)*1000000, J_history_lc_es.shape[0]))
print("Gradient Descent Loop-based Custom (Without Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_lc_nes,3)*1000, round(eta_lc_nes,3)*1000000, J_history_lc_nes.shape[0]))

Gradient Descent Loop-based Custom (Early Stopping) / Execution time 14522.0 [ms]; 14522000.0 [µs] over 7630 iterations
Gradient Descent Loop-based Custom (Without Early Stopping) / Execution time 18780.0 [ms]; 18780000.0 [µs] over 10000 iterations


## Achieved results for the GD implementations

In [21]:
print("Gradient Descent Vectorized (Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_vec_es,3)*1000, round(eta_vec_es,3)*1000000, J_history_vec_es.shape[0]))
print("Gradient Descent Vectorized (Without Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_vec_nes,3)*1000, round(eta_vec_nes,3)*1000000, J_history_vec_nes.shape[0]))
print("\n")
print("Gradient Descent Loop-based Numpy (Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_ln_es,3)*1000, round(eta_ln_es,3)*1000000, J_history_ln_es.shape[0]))
print("Gradient Descent Loop-based Numpy (Without Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_ln_nes,3)*1000, round(eta_ln_nes,3)*1000000, J_history_ln_nes.shape[0]))
print("\n")
print("Gradient Descent Loop-based Custom (Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_lc_es,3)*1000, round(eta_lc_es,3)*1000000, J_history_lc_es.shape[0]))
print("Gradient Descent Loop-based Custom (Without Early Stopping) / Execution time {} [ms]; {} [µs] over {} iterations".format(round(eta_lc_nes,3)*1000, round(eta_lc_nes,3)*1000000, J_history_lc_nes.shape[0]))

Gradient Descent Vectorized (Early Stopping) / Execution time 87.0 [ms]; 87000.0 [µs] over 8000 iterations
Gradient Descent Vectorized (Without Early Stopping) / Execution time 93.0 [ms]; 93000.0 [µs] over 10000 iterations


Gradient Descent Loop-based Numpy (Early Stopping) / Execution time 14149.0 [ms]; 14149000.0 [µs] over 7915 iterations
Gradient Descent Loop-based Numpy (Without Early Stopping) / Execution time 17811.0 [ms]; 17811000.0 [µs] over 10000 iterations


Gradient Descent Loop-based Custom (Early Stopping) / Execution time 14522.0 [ms]; 14522000.0 [µs] over 7630 iterations
Gradient Descent Loop-based Custom (Without Early Stopping) / Execution time 18780.0 [ms]; 18780000.0 [µs] over 10000 iterations


## Comparison between different implementations and sanity check on $\theta$ values

In [22]:
theta = np.zeros((2,1))

alpha = 0.01
num_iters = 10000

theta_vec, Cost_J_vec, eta_gd_vec = gradientDescentVectorized(X, y, np.copy(theta), alpha, num_iters, early = False)
theta_ln, Cost_J_ln, eta_gd_ln = gradientDescentLoopNumpy(X, y, np.copy(theta), alpha, num_iters, early = False)
theta_lc, Cost_J_lc, eta_gd_lc = gradientDescentLoopCustom(X, y, np.copy(theta), alpha, num_iters, early = False)

In [23]:
assert(round(theta_vec[0], 6) == round(theta_ln[0], 6) == round(theta_lc[0], 6))
assert(round(theta_vec[1], 6) == round(theta_ln[1], 6) == round(theta_lc[1], 6))
print('theta found by GD Vectorized : ', theta_vec)
print('theta found by GD Loop-based Numpy : ', theta_ln)
print('theta found by GD Loop-based Custom : ', theta_lc)

theta found by GD Vectorized :  [-3.89578082  1.19303364]
theta found by GD Loop-based Numpy :  [-3.89578082  1.19303364]
theta found by GD Loop-based Custom :  [-3.89578082  1.19303364]


# Benchmarking

* The **iterations** list contains different number of iterations to train the GD algorithm
* Note that each algorithm is executed only ONE time for a fixed iteration number
   * For example: when iteration is 1000, the GD algorithms are called just one time and interally they iterate for 1000 iterations
   * In a more complex and precise case, the entire procedure should be repeated several times and the collected metrics should be averaged; it requires to add an inner-loop that loops for an arbitrary number of iterations

In [24]:
start_iter = 1000
end_iter = 11000
iterations = np.arange(start_iter, end_iter, 1000)
print("Benchmark iterations list: ", iterations)

Benchmark iterations list:  [ 1000  2000  3000  4000  5000  6000  7000  8000  9000 10000]


In [25]:
inner_loop_counter = 10

theta = np.zeros((2,1))

alpha = 0.01

eta_gd_vec_list = []
eta_gd_ln_list = []
eta_gd_lc_list = []

for idx, iteration in enumerate(iterations):
    eta_gd_vec_list_inner = []
    eta_gd_ln_list_inner = []
    eta_gd_lc_list_inner = []
    print("Iteration {} of {} (The GD algorithm runs for {} iterations)".format(idx+1, iterations.shape[0], iterations[idx]))
    for j in range(inner_loop_counter):
        print("Inner Loop Iteration {} of {}".format(j+1, inner_loop_counter))
        print("   > GD Vectorized is running...")
        theta_vec, Cost_J_vec, eta_gd_vec = gradientDescentVectorized(X, y, np.copy(theta), alpha, iteration, early = False)
        print("   > GD Vectorized completed\n")
        print("   > GD Loop-based Numpy is running...")
        theta_ln, Cost_J_ln, eta_gd_ln = gradientDescentLoopNumpy(X, y, np.copy(theta), alpha, iteration, early = False)
        print("   > GD Loop-based Numpy completed\n")
        print("   > GD Loop-based Custom is running...")
        theta_lc, Cost_J_lc, eta_gd_lc = gradientDescentLoopCustom(X, y, np.copy(theta), alpha, iteration, early = False)
        print("   > GD Loop-based Custom completed\n")
        eta_gd_vec_list_inner.append(eta_gd_vec)
        eta_gd_ln_list_inner.append(eta_gd_ln)
        eta_gd_lc_list_inner.append(eta_gd_lc)
        
    
    eta_gd_vec_list.append(np.mean(eta_gd_vec_list_inner))
    eta_gd_ln_list.append(np.mean(eta_gd_ln_list_inner))
    eta_gd_lc_list.append(np.mean(eta_gd_lc_list_inner))

Iteration 1 of 10 (The GD algorithm runs for 1000 iterations)
Inner Loop Iteration 1 of 10
   > GD Vectorized is running...
   > GD Vectorized completed

   > GD Loop-based Numpy is running...
   > GD Loop-based Numpy completed

   > GD Loop-based Custom is running...
   > GD Loop-based Custom completed

Inner Loop Iteration 2 of 10
   > GD Vectorized is running...
   > GD Vectorized completed

   > GD Loop-based Numpy is running...
   > GD Loop-based Numpy completed

   > GD Loop-based Custom is running...
   > GD Loop-based Custom completed

Inner Loop Iteration 3 of 10
   > GD Vectorized is running...
   > GD Vectorized completed

   > GD Loop-based Numpy is running...
   > GD Loop-based Numpy completed

   > GD Loop-based Custom is running...
   > GD Loop-based Custom completed

Inner Loop Iteration 4 of 10
   > GD Vectorized is running...
   > GD Vectorized completed

   > GD Loop-based Numpy is running...
   > GD Loop-based Numpy completed

   > GD Loop-based Custom is running...

KeyboardInterrupt: 

### Express the execution time in milliseconds and microseconds

In [ ]:
# execution time in milliseconds
eta_gd_vec_list_ms = [i*1000 for i in eta_gd_vec_list]
eta_gd_ln_list_ms = [i*1000 for i in eta_gd_ln_list]
eta_gd_lc_list_ms = [i*1000 for i in eta_gd_lc_list]

# execution time in microseconds
eta_gd_vec_list_us = [i*1000000 for i in eta_gd_vec_list]
eta_gd_ln_list_us = [i*1000000 for i in eta_gd_ln_list]
eta_gd_lc_list_us = [i*1000000 for i in eta_gd_lc_list]

## Plot the Benchmark results (in milliseconds)

In [ ]:
plot_scale = 'log'

if (plot_scale == 'linear'):
    # plot in linear scale
    scale = 'linear'
elif (plot_scale == 'log'):
    # plot in semi-log scale (log scale on y-axis)
    scale = 'log'

fig = plt.figure(figsize=(15,6))
plt.scatter(iterations, eta_gd_vec_list_ms, marker = 'x', c = 'r', s = 50, label = 'Gradient Descent Vectorized')
plt.plot(iterations, eta_gd_vec_list_ms, c = 'r')
plt.scatter(iterations, eta_gd_ln_list_ms, marker = 'x', c = 'g', s = 50, label = 'Gradient Descent Loop-based Numpy')
plt.plot(iterations, eta_gd_ln_list_ms, c = 'g')
plt.scatter(iterations, eta_gd_lc_list_ms, marker = 'x', c = 'orange', s = 50, label = 'Gradient Descent Loop-based Custom')
plt.plot(iterations, eta_gd_lc_list_ms, c = 'orange')
plt.yscale(scale)
plt.xlabel('# Iterations', size = 15)
plt.ylabel('Average Computation time [ms]', size = 15)
plt.title('Linear Regression - Benchmarking', size = 15)
plt.legend()
plt.show()

# Find the early stopping iteration for the Gradient Descent implementations

In [ ]:
theta = np.zeros((2,1))

alpha = 0.01
num_iters = 10000

theta_vec_sp, Cost_J_vec_sp, eta_gd_vec_sp = gradientDescentVectorized(X, y, np.copy(theta), alpha, num_iters, early = True)
theta_ln_sp, Cost_J_ln_sp, eta_gd_ln_sp = gradientDescentLoopNumpy(X, y, np.copy(theta), alpha, num_iters, early = True)
theta_lc_sp, Cost_J_lc_sp, eta_gd_lc_sp = gradientDescentLoopCustom(X, y, np.copy(theta), alpha, num_iters, early = True)

In [ ]:
print("Gradient Descent Vectorized with Early stopping stops at {} iterations".format(len(Cost_J_vec_sp)))
print("Gradient Descent Loop-based Numpy with Early stopping stops at {} iterations".format(len(Cost_J_ln_sp)))
print("Gradient Descent Loop-based Custom with Early stopping stops at {} iterations".format(len(Cost_J_lc_sp)))

#### Verifica aggiuntiva early stopping con ulteriori grafici

In [ ]:
theta_vec_es0, J_history_vec_es0, eta_vec_es0 = gradientDescentVectorized(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=10000 ,early = True)
theta_vec_es1, J_history_vec_es1, eta_vec_es1 = gradientDescentVectorized(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=9000 ,early = False)
theta_vec_es2, J_history_vec_es2, eta_vec_es2 = gradientDescentVectorized(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=10000 ,early = False)

print("Gradient Descent Vectorized (Early Stopping) executed in {} [ms] with {} iterations".format(round(eta_vec_es0,3)*1000, J_history_vec_es0.shape[0]))
print("Gradient Descent Vectorized (Without Early Stopping 9000 iterations) executed in {} [ms] with {} iterations".format(round(eta_vec_es1,3)*1000, J_history_vec_es1.shape[0]))
print("Gradient Descent Vectorized (Without Early Stopping 10000 iterations) executed in {} [ms] with {} iterations".format(round(eta_vec_es2,3)*1000, J_history_vec_es2.shape[0]))

#Verifico plottando che nonostante il diverso numero di iterazioni il grafico dell'Early Stopping e quelli senza combacino 
def plotCost(j_history_vec2, label2, j_history_vec1, label1, j_history_vec0, label0):
    fig=plt.figure(figsize=(20,5))
    plt.xlabel('Epochs')
    plt.ylabel('Cost J')
    plt.plot(j_history_vec2, label=label2)
    plt.plot(j_history_vec1, label=label1)
    plt.plot(j_history_vec0, label=label0)
    plt.legend(loc=1)
    #plt.plot(J_history_vec_es1, label="Cost Loop-based (Numpy)")
    #plt.plot(J_history_vec_es2, label="Cost Loop-based (Custom)")

plotCost(J_history_vec_es2, 'Cost Vectorized without Early Stopping with 10000 iterations',J_history_vec_es1, 'Cost Vectorized without Early Stopping with 9000 iterations', J_history_vec_es0, 'Cost Vectorized with Early Stopping')

In [ ]:
#Plot funzione di costo con grafici a barre
def plotBarChart(j_history_vec2, label2, j_history_vec1, label1, j_history_vec0, label0):
    fig=plt.figure(figsize=(20,6))
    plt.ylim(ymin=4.45,ymax=6)
    plt.bar(np.arange(len(j_history_vec0)), j_history_vec0, color='g', label=label0)
    plt.bar(np.arange(len(j_history_vec1)), j_history_vec1, color='b', label=label1)
    plt.bar(np.arange(len(j_history_vec2)), j_history_vec2, color='orange', label=label2)

    plt.legend(loc=1)
    plt.xlabel('Epochs')
    plt.ylabel('Cost J')

plotBarChart(J_history_vec_es0, "Last 5 cost values with Early Stop are: %.15f , %.15f , %.15f , %.15f, %.15f" %(J_history_vec_es0[-5], J_history_vec_es0[-4], J_history_vec_es0[-3], J_history_vec_es0[-2], J_history_vec_es0[-1]),
            J_history_vec_es1, "Last 5 cost values without Early Stopping with 9000 iterations are: %.15f , %.15f , %.15f , %.15f, %.15f" %(J_history_vec_es1[-5], J_history_vec_es1[-4], J_history_vec_es1[-3], J_history_vec_es1[-2], J_history_vec_es1[-1]),
            J_history_vec_es2, "Last 5 cost values without Early Stopping with 10000 iterations are: %.15f , %.15f , %.15f , %.15f, %.15f" %(J_history_vec_es2[-5], J_history_vec_es2[-4], J_history_vec_es2[-3], J_history_vec_es2[-2], J_history_vec_es2[-1]))

# Benchmarking the inner loop iteration time

* Move the **start** and **end** time statements inside the iteration loop

In [ ]:
def gradientDescentVectorizedIter(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=1500, early = False):
    m = y.size
    J_history = np.zeros(num_iters)
    eta = []
    
    for iter in np.arange(num_iters):
        start = time.time()
        h = X.dot(theta)
        
        # ! simoultaneusly update all the parameters 
        # vectorized implementation
        theta = theta - alpha*(1/m)*(X.T.dot(h-y))
        J_history[iter], _ = computeCostVectorized(X, y, theta)
        
        # early stopping
        if (early == True) & (J_history[iter] == J_history[iter-1]):
            break
            
        end = time.time()
        eta.append(end - start)
    return(theta.ravel(), J_history[J_history != 0], eta)

In [ ]:
def gradientDescentLoopNumpyIter(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=1500, early = False):
    m = y.size
    J_history = np.zeros(num_iters)
    eta = []
    
    for iter in np.arange(num_iters):
        start = time.time()
        
        h = np.zeros((m,1))
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                h[i] += theta[j] * X[i,j]

        # ! simoultaneusly update all the parameters
        for j in range(X.shape[1]):
            gradient = 0
            for i in range(X.shape[0]):
                gradient += (h[i]-y[i])*X[i,j]
            theta[j] = theta[j] - alpha*(1/m)*gradient

        J_history[iter], _ = computeCostLoopNumpy(X, y, theta)
        
        # early stopping
        if (early == True) & (J_history[iter] == J_history[iter-1]):
            break
            
        end = time.time()
        eta.append(end - start)
    return(theta.ravel(), J_history[J_history != 0], eta)

In [ ]:
def gradientDescentLoopCustomIter(X, y, theta=np.zeros((2,1)), alpha=0.01, num_iters=1500, early = False):
    m = y.size
    J_history = np.zeros(num_iters)
    eta = []
    
    for iter in np.arange(num_iters):
        start = time.time()
        
        h = np.zeros((m,1))
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                h[i] += theta[j] * X[i,j]

        # ! simoultaneusly update all the parameters
        for j in range(X.shape[1]):
            gradient = 0
            for i in range(X.shape[0]): 
                gradient += (h[i]-y[i])*X[i,j]
            theta[j] = theta[j] - alpha*(1/m)*gradient

        J_history[iter], _ = computeCostLoopCustom(X, y, theta)
        
        # early stopping
        if (early == True) & (J_history[iter] == J_history[iter-1]):
            break
            
        end = time.time()
        eta.append(end - start)
    return(theta.ravel(), J_history[J_history != 0], eta)

In [ ]:
theta = np.zeros((2,1))

alpha = 0.01
num_iters = 10000

_, _, eta_vec_iter = gradientDescentVectorizedIter(X, y, np.copy(theta), alpha, num_iters, early = False)
_, _, eta_ln_iter = gradientDescentLoopNumpyIter(X, y, np.copy(theta), alpha, num_iters, early = False)
_, _, eta_lc_iter = gradientDescentLoopCustomIter(X, y, np.copy(theta), alpha, num_iters, early = False)

mean_eta_vec_iter = np.mean(eta_vec_iter)
mean_eta_ln_iter = np.mean(eta_ln_iter)
mean_eta_lc_iter = np.mean(eta_lc_iter)

In [ ]:
print("Gradient Descent Vectorized : {} [ms]/iteration; {} [µs]/iteration (Average over {} iterations)".format(round(mean_eta_vec_iter, 6)*1000, round(mean_eta_vec_iter, 6)*1000000, num_iters))
print("Gradient Descent Loop-based Numpy : {} [ms]/iteration; {} [µs]/iteration (Average over {} iterations)".format(round(mean_eta_ln_iter, 6)*1000, round(mean_eta_ln_iter, 6)*1000000, num_iters))
print("Gradient Descent Loop-based Custom : {} [ms]/iteration; {} [µs]/iteration (Average over {} iterations)".format(round(mean_eta_lc_iter, 6)*1000, round(mean_eta_lc_iter, 6)*1000000, num_iters))

# Benchmarking n1: Gradient Descent Vectorized Vs Normal Equation (Custom)

## Normal Equation (Custom)

$$\large \theta = (X^TX)^{-1}X^Ty$$

In [ ]:
def normalEquations(X, y):
    start = time.time()
    #theta = np.dot(np.dot(np.linalg.pinv(np.dot(np.transpose(X), X)), np.transpose(X)), y)
    pinv = np.linalg.pinv(X.T.dot(X))
    theta_ne = pinv.dot(X.T).dot(y)
    end = time.time()
    eta_ne = end-start
    return theta_ne.ravel(), eta_ne

In [ ]:
theta = np.zeros((2,1))

alpha = 0.01
num_iters = 10000
num_iters_gd_vs_ne = 100

eta_gd_bench_list_1 = []
eta_ne_bench_list_1 = []

for i in range(num_iters_gd_vs_ne):
    print("Iteration {} of {}".format(i+1, num_iters_gd_vs_ne))
    _, _, eta_gd_vec_1 = gradientDescentVectorized(X, y, np.copy(theta), alpha, num_iters, early = False)
    _, eta_ne_1 = normalEquations(X, y)
    eta_gd_bench_list_1.append(eta_gd_vec_1)
    eta_ne_bench_list_1.append(eta_ne_1)
    
mean_eta_gd_bench_1 = np.mean(eta_gd_bench_list_1)
mean_eta_ne_bench_1 = np.mean(eta_ne_bench_list_1)

In [ ]:
print("Average execution time Gradient Descent Vectorized : {} [ms] {} [µs]".format(round(mean_eta_gd_bench_1, 6)*1000, round(mean_eta_gd_bench_1, 6)*1000000))
print("Average execution time Normal Equation : {} [ms] {} [µs]".format(round(mean_eta_ne_bench_1, 6)*1000, round(mean_eta_ne_bench_1, 6)*1000000))

# Benchmarking n2: Normal Equation (Custom) Vs Linear Regression (Sklearn)

In [ ]:
def LR(X, y):
    start = time.time()
    regr = LinearRegression()
    regr.fit(X,y)
    end = time.time()
    eta_lr = end - start
    return regr, eta_lr

In [ ]:
num_iters_ne_vs_lr = 100

eta_ne_bench_list_2 = []
eta_lr_bench_list_2 = []

for i in range(num_iters_ne_vs_lr):
    print("Iteration {} of {}".format(i+1, num_iters_ne_vs_lr))
    _, eta_ne_2 = normalEquations(X, y)
    _, eta_lr_2 = LR(X, y)
    
    eta_ne_bench_list_2.append(eta_ne_2)
    eta_lr_bench_list_2.append(eta_lr_2)
    
mean_eta_ne_bench_2 = np.mean(eta_ne_bench_list_2)
mean_eta_lr_bench_2 = np.mean(eta_lr_bench_list_2)

In [ ]:
print("Average execution time Normal Equation : {} [ms] {} [µs]".format(round(mean_eta_ne_bench_2, 6)*1000, round(mean_eta_ne_bench_2, 6)*1000000))
print("Average execution time Linear Regression (Sklearn) : {} [ms] {} [µs]".format(round(mean_eta_lr_bench_2, 6)*1000, round(mean_eta_lr_bench_2, 6)*1000000))